In [56]:
from IPython.core.magic import register_cell_magic
from IPython.display import Markdown
import datetime
from datetime import date
import glob
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import warnings
import seaborn as sns

In [57]:
columns = ['CVE']
metasploit_df = pd.read_csv('metasploit.txt', header=None, names=columns)

,CVE
0,CVE-2007-4387
1,CVE-2014-6041
2,CVE-2005-2611
3,CVE-2005-0771
4,CVE-2004-0795
...,...
2681,CVE-2022-29847
2682,CVE-2022-29848
2683,CVE-2021-36934
2684,CVE-2011-3402


In [58]:
CISA_df = pd.read_csv('known_exploited_vulnerabilities.csv')
CISA_df = CISA_df.rename(columns={"cveID": "CVE"})

,CVE,vendorProject,product,vulnerabilityName,dateAdded,shortDescription,requiredAction,dueDate,notes
0,CVE-2021-27104,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,NaN
1,CVE-2021-27102,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,NaN
2,CVE-2021-27101,Accellion,FTA,Accellion FTA SQL Injection Vulnerability,2021-11-03,Accellion FTA contains a SQL injection vulnera...,Apply updates per vendor instructions.,2021-11-17,NaN
3,CVE-2021-27103,Accellion,FTA,Accellion FTA Server-Side Request Forgery (SSR...,2021-11-03,Accellion FTA contains a server-side request f...,Apply updates per vendor instructions.,2021-11-17,NaN
4,CVE-2021-21017,Adobe,Acrobat and Reader,Adobe Acrobat and Reader Heap-based Buffer Ove...,2021-11-03,Acrobat Acrobat and Reader contain a heap-base...,Apply updates per vendor instructions.,2021-11-17,NaN
...,...,...,...,...,...,...,...,...,...
977,CVE-2023-35078,Ivanti,Endpoint Manager Mobile (EPMM),Ivanti Endpoint Manager Mobile Authentication ...,2023-07-25,"Ivanti Endpoint Manager Mobile (EPMM, previous...",Apply mitigations per vendor instructions or d...,2023-08-15,https://forums.ivanti.com/s/article/CVE-2023-3...
978,CVE-2023-38606,Apple,Multiple Products,Apple Multiple Products Kernel Unspecified Vul...,2023-07-26,"Apple iOS, iPadOS, macOS, tvOS, and watchOS co...",Apply mitigations per vendor instructions or d...,2023-08-16,"https://support.apple.com/en-us/HT213841, http..."
979,CVE-2023-37580,Zimbra,Collaboration (ZCS),Zimbra Collaboration (ZCS) Cross-Site Scriptin...,2023-07-27,Zimbra Collaboration Suite (ZCS) contains a cr...,Apply mitigations per vendor instructions or d...,2023-08-17,https://wiki.zimbra.com/wiki/Security_Center
980,CVE-2023-35081,Ivanti,Endpoint Manager Mobile (EPMM),Ivanti Endpoint Manager Mobile (EPMM) Path Tra...,2023-07-31,Ivanti Endpoint Manager Mobile (EPMM) contains...,Apply mitigations per vendor instructions or d...,2023-08-21,https://forums.ivanti.com/s/article/CVE-2023-3...


In [59]:
epss_df = pd.read_csv('epss_scores-current.csv', skiprows=1)
epss_df = epss_df.rename(columns={"cve": "CVE"})
epss_df = epss_df[epss_df.epss > .90]

,CVE,epss,percentile
4,CVE-1999-0005,0.94040,0.98793
40,CVE-1999-0042,0.92913,0.98638
65,CVE-1999-0067,0.93322,0.98691
68,CVE-1999-0070,0.96090,0.99260
110,CVE-1999-0113,0.93324,0.98692
...,...,...,...
204393,CVE-2023-28771,0.91881,0.98515
206677,CVE-2023-33246,0.96386,0.99348
207269,CVE-2023-34362,0.94054,0.98796
207546,CVE-2023-35078,0.95533,0.99111


In [60]:
epss_cves = epss_df['CVE']
metasploit_cves = metasploit_df['CVE']
CISA_cves = CISA_df['CVE']

In [61]:
CVE_list = pd.merge(CISA_cves, metasploit_cves, how='outer', left_on='CVE', right_on='CVE')
CVE_list = pd.merge(CVE_list, epss_cves, how='outer', left_on='CVE', right_on='CVE')
CVE_list = CVE_list.drop_duplicates()


,CVE
0,CVE-2021-27104
1,CVE-2021-27102
2,CVE-2021-27101
3,CVE-2021-27103
4,CVE-2021-21017
...,...
5519,CVE-2022-34753
5520,CVE-2022-35871
5521,CVE-2022-40881
5522,CVE-2022-42233


In [64]:
row_accumulator = []
for filename in glob.glob('nvd.jsonl'):
    with open(filename, 'r', encoding='utf-8') as f:
        nvd_data = json.load(f)
        for entry in nvd_data:
            cve = entry['cve']['id']
            try:
                assigner = entry['cve']['sourceIdentifier']
            except KeyError:
                assigner = 'Missing_Data'
            try:
                published_date = entry['cve']['published']
            except KeyError:
                published_date = 'Missing_Data'
            try:
                attack_vector = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['attackVector']
            except KeyError:
                attack_vector = 'Missing_Data'
            try:
                attack_complexity = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['attackComplexity']
            except KeyError:
                attack_complexity = 'Missing_Data'
            try:
                privileges_required = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['privilegesRequired']
            except KeyError:
                privileges_required = 'Missing_Data'
            try:
                user_interaction = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['userInteraction']
            except KeyError:
                user_interaction = 'Missing_Data'
            try:
                scope = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['scope']
            except KeyError:
                scope = 'Missing_Data'
            try:
                confidentiality_impact = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['confidentialityImpact']
            except KeyError:
                confidentiality_impact = 'Missing_Data'
            try:
                integrity_impact = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['integrityImpact']
            except KeyError:
                integrity_impact = 'Missing_Data'
            try:
                availability_impact = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['availabilityImpact']
            except KeyError:
                availability_impact = 'Missing_Data'
            try:
                base_score = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['baseScore']
            except KeyError:
                base_score = '0.0'
            try:
                base_severity = entry['cve']['metrics']['cvssMetricV31'][0]['cvssData']['baseSeverity']
            except KeyError:
                base_severity = 'Missing_Data'
            try:
                exploitability_score = entry['cve']['metrics']['cvssMetricV31'][0]['exploitabilityScore']
            except KeyError:
                exploitability_score = 'Missing_Data'
            try:
                impact_score = entry['cve']['metrics']['cvssMetricV31'][0]['impactScore']
            except KeyError:
                impact_score = 'Missing_Data'
            try:
                cwe = entry['cve']['weaknesses'][0]['description'][0]['value']
            except KeyError:
                cwe = 'Missing_Data'
            try:
                description = entry['cve']['descriptions'][0]['value']
            except IndexError:
                description = ''
            new_row = {
                'CVE': cve,
                'Published': published_date,
                'CVSS Score': base_score,
                'Description': description
            }
            if not description.startswith('** REJECT **'): # disputed, rejected and other non issues start with '**'
                row_accumulator.append(new_row)
        nvd = pd.DataFrame(row_accumulator)
        
nvd['Published'] = pd.to_datetime(nvd['Published'])
nvd = nvd.sort_values(by=['Published'])
nvd = nvd.reset_index(drop=True)

,CVE,Published,CVSS Score,Description
0,CVE-1999-0095,1988-10-01 04:00:00.000,0.0,"The debug command in Sendmail is enabled, allo..."
1,CVE-1999-0082,1988-11-11 05:00:00.000,0.0,CWD ~root command in ftpd allows root access.
2,CVE-1999-1471,1989-01-01 05:00:00.000,0.0,Buffer overflow in passwd in BSD based operati...
3,CVE-1999-1122,1989-07-26 04:00:00.000,0.0,Vulnerability in restore in SunOS 4.0.3 and ea...
4,CVE-1999-1467,1989-10-26 04:00:00.000,0.0,Vulnerability in rcp on SunOS 4.0.x allows rem...
...,...,...,...,...
209006,CVE-2023-38940,2023-08-07 19:15:11.610,0.0,"Tenda F1203 V2.0.1.6, FH1203 V2.0.1.6 and FH12..."
209007,CVE-2023-39349,2023-08-07 19:15:11.697,8.1,Sentry is an error tracking and performance mo...
209008,CVE-2023-39363,2023-08-07 19:15:11.873,5.9,Vyer is a Pythonic Smart Contract Language for...
209009,CVE-2023-39550,2023-08-07 19:15:11.987,0.0,"Netgear JWNR2000v2 v1.0.0.11, XWN5001 v0.4.1.1..."


In [72]:
patchthisapp_df = pd.merge(CVE_list, nvd, how='inner', left_on='CVE', right_on='CVE')
patchthisapp_df = pd.merge(patchthisapp_df, epss_df, how='inner', left_on='CVE', right_on='CVE')
patchthisapp_df = patchthisapp_df[['CVE', 'CVSS Score', 'epss', 'Description', 'Published']]
patchthisapp_df = patchthisapp_df.rename(columns={"epss": "EPSS"})
patchthisapp_df.to_csv('data/data.csv', index=False)
patchthisapp_df


,CVE,CVSS Score,EPSS,Description,Published
0,CVE-2018-4939,9.8,0.97236,Adobe ColdFusion Update 5 and earlier versions...,2018-05-19 17:29:01.480
1,CVE-2018-15961,0.0,0.97474,Adobe ColdFusion versions July 12 release (201...,2018-09-25 13:29:01.567
2,CVE-2018-4878,9.8,0.97442,A use-after-free vulnerability was discovered ...,2018-02-06 21:29:00.347
3,CVE-2017-9805,0.0,0.97547,The REST Plugin in Apache Struts 2.1.1 through...,2017-09-15 19:29:00.237
4,CVE-2021-42013,9.8,0.97515,It was found that the fix for CVE-2021-41773 i...,2021-10-07 16:15:09.270
...,...,...,...,...,...
3460,CVE-2022-34753,8.8,0.97005,A CWE-78: Improper Neutralization of Special E...,2022-07-13 21:15:08.163
3461,CVE-2022-35871,7.8,0.96201,This vulnerability allows remote attackers to ...,2022-07-25 19:15:45.637
3462,CVE-2022-40881,9.8,0.94859,SolarView Compact 6.00 was discovered to conta...,2022-11-17 04:15:10.857
3463,CVE-2022-42233,9.8,0.90170,Tenda 11N with firmware version V5.07.33_cn su...,2022-10-20 17:15:10.617
